In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from IPython.core.display import HTML, Image, display

In [ ]:
from timeit import default_timer as t_now
import time
import numpy as np
import dask
import dask.distributed
import dask.array as da

def to_rgba_u32(r: np.ndarray, 
                g: np.ndarray, 
                b: np.ndarray, 
                clamp: int = 3000, 
                nodata: int = -999):
    a = (r != nodata).astype('uint32')*(0xFF000000)
    r,g,b = (np.clip(x, 0, clamp).astype('uint32')*255//clamp 
             for x in (r,g,b))
    
    # 4-byte uint containing |a,b,g,r| where a is MSB
    out = a
    out[:] |= r
    out[:] |= g<<8
    out[:] |= b<<16
    
    return out

def u32_to_rgba(x):
    return x.view('uint8').reshape(*x.shape, 4)

In [ ]:
client = dask.distributed.Client(n_workers=1, 
                                 threads_per_worker=32, 
                                 processes=True, 
                                 ip='127.0.0.1')
client

### Alternative Client Setup

Connect to externally launched cluster:

```python
client = dask.distributed.Client('127.0.0.1:8786')
```

Local cluster launched as following:

```bash
# Scheduler
/opt/dea/envs/dask/bin/dask-scheduler --interface lo

# Worker ovesubscribed 8 threads per core
/opt/dea/envs/dask/bin/dask-worker --interface lo \
 --memory-limit 24G \
 --nthreads 32 \
 --nprocs 1 \
 tcp://127.0.0.1:8786
```

In [ ]:
# pip install https://github.com/Kirill888/wk-misc/releases/download/v1.0/kk_dtools-1-py3-none-any.whl
from kk.dtools import rio_activate, rio_getenv

settings = rio_activate(client, aws=dict(region_name='ap-southeast-2'))
display(settings[0])

In [ ]:
def cluster_rio_info(client):
    oo = rio_getenv(client)
    return oo[0]

cluster_rio_info(client)

In [ ]:
import datacube
from datacube.api.core import get_bounds, output_geobox
from datacube.utils.geometry import BoundingBox, bbox_union, CRS
from datacube.testutils.io import rio_slurp
from datacube.storage import measurement_paths

PRODUCT = 'ls8_nbart_geomedian_annual'
NATIVE_RES = (-25, 25)
CRS = 'EPSG:3577'
dc = datacube.Datacube(env='gm')

In [ ]:
%%time
dss = dc.find_datasets(product=PRODUCT,
                       time=('2017-01-01', '2017-12-31'))

display(HTML('''
Found: <b>{}</b> datasets <br>
CRS: <b>{}</b>
'''.format(
    len(dss), 
    str(dss[0].crs)
)))

In [ ]:
%%time
xx = dc.load(product=PRODUCT,
             datasets=dss,
             output_crs=CRS,
             group_by='solar_day',
             #skip_broken_datasets=True,
             resolution=tuple(32*n for n in NATIVE_RES),
             dask_chunks={'x': 256, 'y': 256, 'time': 1})


display(xx)
dd = xx.red.data.to_delayed()
display(HTML('''
<bl>
Number of chunks per band: <b>{}x{}x{}</b>
'''.format(*dd.shape)
))

In [ ]:
%%time
cc32 = da.map_blocks(to_rgba_u32, 
                     xx.red.data, xx.green.data, xx.blue.data, 
                     clamp=3000, nodata=-999, 
                     dtype='uint32')

display(cc32)

In [ ]:
n_image_files = len(dss)*3

display(HTML('''
Launching compute for <b>{:,d} x {:,d}</b> raster 
split across <b>{:,d}</b> files
'''.format(*cc32.shape[-2:], n_image_files)))

In [ ]:
t0 = t_now()
fut = client.compute(cc32)  # Launch computation, but don't block

n_wait = 0
while fut.done() is False:
    time.sleep(0.1)
    n_wait += 1
    if n_wait % 10 == 0:
        print('.', end='', flush=True)
    if n_wait % 100 == 0:
        t_elapsed = t_now() - t0
        print(': {:.3f} secs'.format(t_elapsed))

cc32_ = fut.result()
t_elapsed = t_now() - t0

print("\nTotal load time: {:.3f} secs".format(t_elapsed))

In [ ]:
display(HTML("""
Loaded <b>{:,d}</b> files in <b>{:.3f}</b> secs, <b>{:.1f}</b> files per second
""".format(n_image_files, t_elapsed, n_image_files/t_elapsed)))

In [ ]:
cc8 = u32_to_rgba(cc32_[0,:,:])
display(HTML('Saving <b>{:,d} x {:,d}</b> image to disk'.format(*cc8.shape[:2])))

In [ ]:
%%time
display(HTML('JPEG'))
plt.imsave('gm-ls8-overview.jpg', cc8)

In [ ]:
%%time
display(HTML('PNG'))
plt.imsave('gm-ls8-overview.png', cc8)

In [ ]:
display(HTML('''
Download image: 
<a href="gm-ls8-overview.jpg" download>jpeg</a>
<a href="gm-ls8-overview.png" download>png</a>

<img src="gm-ls8-overview.jpg" style="width:1024px">
'''))

--------------------------------------

In [ ]:
mask = cc8[:,:,3]>0
r,g,b = (int(v) for v in 
         np.round(cc8[mask,:3].mean(axis=0)))

display(HTML('''
<div>Average Color</div>
#{r:02X}{g:02X}{b:02X}
<div style="width: 200px; 
height:200px;
background:rgb({r:},{g:},{b:});"></div>
'''.format(r=r,g=g,b=b)))

----------------------------------------------